In [1]:
import sleqp

In [2]:
import numpy as np
import math

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

# Least-squares problems

SLEQP has built-in support for nonlinear least-squares problems. As an example, we can formulate the Rosenbrock problem as a nonlinear least-squares problem. Recall that the original objective is given by

$$
f(x, y) := (a-x)^2 + b(y-x^2)^2,
$$

which corresponds to minimizing $\tfrac{1}{2} \|r(x, y)\|^{2}$, where

$$
r(x, y) := (a - x, \sqrt{b} (y - x^{2}))
$$

## Interfaces

In order to compute EQP steps with respect to a residual function $r : \mathbb{R}^{n} \to \mathbb{R}^{k}$, `SLEQP` uses the Gauss-Newton approximation of the Hessian of $\tfrac{1}{2} \|r(x, y)\|^{2}$, given by $J_r^{T} J_r$, requring only first-order information. Due to the iterative nature of the underlying
alagorithms, we only require products $J_r d_f$ and $d_{a}^{T} J_r$, where $d_f \in \mathbb{R}^{n}$ is a forward direction and $d_a \in \mathbb{R}^{k}$ is and adjoint direction. If the complete Jacobian $J_r$ is computed, this corresponds to matrix-vector products. It is however sometimes simpler to compute the products directly, in particular, if $J_r$ is too large to fit into memory, or if the Gauss-Newton converges fewer than $\min(k, n)$ steps. Notably, automatic differentiation software, such as [ADOL-C](https://github.com/coin-or/ADOL-C) often provides interfaces to compute forward / adjoint derivatives.

In [4]:
class RosenbrockLSQFunc:

  def __init__(self):
    self.a = 1.
    self.b = 100.

  def set_value(self, values, reason):
    self.values = values

  def lsq_residuals(self):
    x = self.values[0]
    y = self.values[1]

    return np.array([self.a - x,
                     math.sqrt(self.b) * (y - (x * x))])


  def lsq_jac_forward(self, forward_direction):
    x = self.values[0]
    y = self.values[1]

    dx = forward_direction[0]
    dy = forward_direction[1]

    return np.array([-1. * dx,
                     math.sqrt(self.b)*(-2.*x*dx + dy)])


  def lsq_jac_adjoint(self, adjoint_direction):
    x = self.values[0]
    y = self.values[1]

    dx = adjoint_direction[0]
    dy = adjoint_direction[1]

    return np.array([-1.*dx - 2*math.sqrt(self.b)*x*dy,
                     math.sqrt(self.b)*(dy)])


In [5]:
var_lb = np.array([-np.inf, -np.inf])
var_ub = np.array([np.inf, np.inf])

x0 = np.array([0., 0.])

num_residuals = 2

In [6]:
func = RosenbrockLSQFunc()

In [7]:
problem = sleqp.LSQProblem(func,
                           num_residuals,
                           var_lb,
                           var_ub)

In [8]:
solver = sleqp.Solver(problem, x0)

In [9]:
solver.solve()

INFO:sleqp:Solving a problem with 2 variables, 0 constraints, 0 Jacobian nonzeros
INFO:sleqp: Iteration |          Merit  val |       Obj val |      Feas res |     Slack res |      Stat res |       Penalty |   Working set |         LP tr |        EQP tr |   Primal step |     Dual step |          Step type
INFO:sleqp:         0 |    5.0000000000e-01 |  5.000000e-01 |  0.000000e+00 |               |               |  1.000000e+01 |               |               |               |               |               |                   
INFO:sleqp:         1 |    5.0000000000e-01 |  5.000000e-01 |  0.000000e+00 |  0.000000e+00 |  1.000000e+00 |  1.000000e+01 |            -- |  5.656854e-01 |  1.000000e+00 |  1.000000e+00 |  0.000000e+00 |           Rejected
INFO:sleqp:         2 |    5.0000000000e-01 |  5.000000e-01 |  0.000000e+00 |  0.000000e+00 |  1.000000e+00 |  1.000000e+01 |            -- |  5.000000e-01 |  5.000000e-01 |  5.000000e-01 |  0.000000e+00 |           Rejected
INFO:sleqp:       

In [10]:
solution = solver.solution

In [11]:
solution.primal

array([1., 1.])